In [ ]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

intents_path = '/content/drive/MyDrive/Colab Notebooks/chatbot-flask-simple/data/intents'

# inference model variables
inference_load_intents_from = os.path.join(intents_path, 'intents_job_intents.json')

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(inference_load_intents_from, encoding='cp1252').read()
intents = json.loads(data_file)

for intent in intents['intents']:
     for pattern in intent['patterns']:
          w = nltk.word_tokenize(pattern)
          words.extend(w)

          documents.append((w, intent['tag']))

          if intent['tag'] not in classes:
               classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique lemmatized words', words)

pickle.dump(words,open(os.path.join(intents_path, 'intents_words.pkl'),'wb'))
pickle.dump(classes,open(os.path.join(intents_path, 'intents_classes.pkl'),'wb'))

# init training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
     bag = []
     pattern_words = doc[0]
     pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

     for w in words:
          bag.append(1) if w in pattern_words else bag.append(0)

     output_row = list(output_empty)
     output_row[classes.index(doc[1])] = 1

     training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists.  X - patterns, y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print('Training data created')

# Create model with 3 layers.  First layer 128 neurons, second layer 64 neurons
# and 3rd output layer contains number of neurons equal to number of intents to
# predict
# output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model.  Stochastic gradient descent with Nesterov accelerated
# gradient gives good
# results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
model.save(os.path.join(intents_path, 'intents_chatbot_model.h5'), hist)

print('model created')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
61 documents
11 classes ['compliment', 'feeling', 'goodbye', 'greeting', 'manager', 'name', 'package_tracking', 'profane', 'return_product', 'thanks', 'tracking']
88 unique lemmatized words ["'", "'s", ',', '.', '0983834298342341', '1234509873234323', '1983-2343-2343-2343', 'a', 'am', 'are', 'back', 'broken', 'bye', 'can', 'computer', 'dead', 'device', 'die', 'do', 'doing', 'drop', 'equipment', 'feel', 'feeling', 'fix', 'fixed', 'get', 'give', 'go', 'going', 'good', 'goodbye', 'hate', 'have', 'hear', 'hello', 'help', 'helpful', 'here', 'hi', 'how', 'i', 'is', 'it', 'later', 'like', 'long', 'manager', 'me', 'money', 'my', 'name', 'need', 'not', 'now', 'package', 'product', 'refund', 'report', 'return', 'see', 'shipment', 'some', 'something', 'speak', 'speaking', 's

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


13/13 [==============================] - 0s 2ms/step - loss: 2.3576 - accuracy: 0.1311
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 2.1570 - accuracy: 0.3115
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.9980 - accuracy: 0.3279
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.7909 - accuracy: 0.3607
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.7581 - accuracy: 0.4098
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.6196 - accuracy: 0.5246
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.5520 - accuracy: 0.4754
Epoch 8/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.5590 - accuracy: 0.4918
Epoch 9/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.3649 - accuracy: 0.5902
Epoch 10/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.2826 - accuracy: 0.5410
Epoch 11/100